In [2]:
import string
import itertools
import os
import subprocess
import random
import string
import re
import hashlib

# up to pattern size of 19, dunno why
# naive and naive_spec on lager patterns
# low level reader


def randomString(stringLength=256, subject=True):
    """Generate a random string of fixed length """
    letters = "ab"
    if subject:
        chunk = 1024*1024*128 # 128MB
        if os.path.exists("./data/subject.txt"):
          os.remove("./data/subject.txt")
        with open("./data/subject.txt","a+") as s:
            for j in range(0,int(((stringLength+chunk-1)/chunk))):
                    rightBound = (j+1)*chunk if (j+1) * chunk < stringLength else stringLength
                    randomStr = ''.join(random.choice(letters) for i in range(j*chunk,rightBound))
                    s.write(randomStr)
        return
    else:
        randomStr =  ''.join(random.choice(letters) for i in range(stringLength))
        with open("./data/pattern.txt","w") as s:
            s.write(randomStr)
        return randomStr


files = [
    
#         "r_naive_spec_chunk.txt",
#         "r_naive_spec_nochunk.txt",
        "clean_pe_nochunk.txt",
        "clean_pe_chunk.txt",
        "cuda_naive_chunk.txt",
        "cuda_naive_nochunk.txt",
        "cuda_naivec_nochunk.txt",
        "cuda_naivec_chunk.txt",
        "cuda_naivesh_chunk.txt",
        "cuda_naivesh_nochunk.txt",
        "cuda_kmp_chunk.txt",
        
        
        ]

def match(pattern,filename,size = 0, offset = 0):
    commands = [
#     f"./fun {pattern} dirty 0 > dirty_chunk.txt",
#     f"./fun {pattern} dirty 1 > dirty_nochunk.txt",
#     f"./fun {pattern} dirtynope 0 > dirty_nope_chunk.txt",
#     f"./fun {pattern} dirtynope 1 > dirty_nope_nochunk.txt",
#     f"./fun {pattern} kmp 0 > kmp_impala_chunk_spec.txt",
#     f"./fun {pattern} cleankmp 0 > kmp_impala_chunk.txt",
#     f"./fun {pattern} kmp 1 > kmp_pe_nochunk.txt",
#     f"./fun {pattern} dirtynaive 0 > dirty_naive_chunk.txt",
#     f"./fun {pattern} dirtynaive 1 > dirty_naive_nochunk.txt",
#     f"./fun {pattern} cleanpe 0 > clean_pe_chunk.txt",
#     f"./fun --pattern={pattern} --filename={filename} --algorithm=r_naive_spec --type=0 --size={size} --offset={offset} > r_naive_spec_chunk.txt",    
#     f"./fun --pattern={pattern} --filename={filename} --algorithm=r_naive_spec --type=1 --size={size} --offset={offset} > r_naive_spec_nochunk.txt",    
    f"./fun --pattern={pattern} --filename={filename} --algorithm=cleanpe --type=1 --size={size} --offset={offset} > clean_pe_nochunk.txt",
    f"./fun --pattern={pattern} --filename={filename} --algorithm=cleanpe --type=0 --size={size} --offset={offset} > clean_pe_chunk.txt",
#     f"./fun {pattern} clean 0 > clean_chunk.txt",
#     f"./fun {pattern} clean 1 > clean_nochunk.txt",
#     f"./fun {pattern} cleanjit 0 > clean_jitc.txt",
#     f"./fun {pattern} cleanjit 1 > clean_jitnc.txt",
    
#     f"./clean.out {pattern} naive 0 > naive_cuda_chunk.txt",
    f"./clean.out --pattern={pattern} --filename={filename} --algorithm=naive --type=0 --size={size} --offset={offset} > cuda_naive_chunk.txt",
    f"./clean.out --pattern={pattern} --filename={filename} --algorithm=naive --type=1 --size={size} --offset={offset} > cuda_naive_nochunk.txt",
    f"./clean.out --pattern={pattern} --filename={filename} --algorithm=naivec --type=1 --size={size} --offset={offset} > cuda_naivec_nochunk.txt",
    f"./clean.out --pattern={pattern} --filename={filename} --algorithm=naivec --type=0 --size={size} --offset={offset} > cuda_naivec_chunk.txt",
    f"./clean.out --pattern={pattern} --filename={filename} --algorithm=naivesh --type=1 --size={size} --offset={offset} > cuda_naivesh_nochunk.txt",
    f"./clean.out --pattern={pattern} --filename={filename} --algorithm=naivesh --type=0 --size={size} --offset={offset} > cuda_naivesh_chunk.txt", 
    f"./clean.out --pattern={pattern} --filename={filename} --algorithm=kmp --type=0 --size={size} --offset={offset} > cuda_kmp_chunk.txt", 
    
#     f"./clean.out {pattern} kmp 0 > kmp_cuda_chunk.txt",
#     f"./clean.out {pattern} kmp 1 > kmp_nochunk.txt",
        
#     f"./clean.out {pattern} const 1 > const_nochunk.txt",
#     f"./clean.out {pattern} naivesh 1 > shared_nochunk.txt"
        
        
    ]
    cmd = ""
    
    for command in commands[:int(len(commands)/2)]:    
        cmd = cmd + " && " + command
    cmd = cmd[4:]
    os.system(cmd)
    cmd = ""
    
    for command in commands[int(len(commands)/2):]:    
        cmd = cmd + " && " + command
    cmd = cmd[4:]
    os.system(cmd)

                
def generate_strings(length=3):
#     chars = string.ascii_letters + string.digits  # "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    chars = 'ab'
    for item in itertools.product(chars, repeat=length):
        yield "".join(item)
       
# cartesian product of input iterables
# strings = [''.join(x) for x in itertools.product('abcdefghijklmnopqrstuvwxyz', repeat=20)]
# patterns = [''.join(x) for x in itertools.product('abcdefghijklmnopqrstuvwxyz', repeat=6)]

strings = generate_strings(15)
patterns = generate_strings(5)

def test_correctness(f):
    for num in range(6,10):
        strings = generate_strings(num)
        for i in strings:
            for num2 in range(1,5):
                patterns = generate_strings(num2)
                for j in patterns:
                    res1 = match(j,i)
                    res2 = match2(j,i,f)
                    if(res1 == res2):
                        continue
                    else:
                        error = f"Test failed on: pattern = {j} string = {i}"
                        print(error)
                        return
    print("Test passed")

# with open("output.txt") as f:
#     test(f)

def all_equal(iterable):
    "Returns True if all the elements are equal to each other"
    g = itertools.groupby(iterable)
    return next(g, True) and not next(g, False)

def test():
    rePatt = re.compile('running time [\d]*\.?[\d]*')
    eq = True
    ofiles = [open(file,'r') for file in files]
    outputs = ["" for f in ofiles]
    mask = [False for i in ofiles]
    def innerLoop1(f):
        while(rePatt.findall(f.readline()) == []):
            pass
        
    for f in ofiles:
            innerLoop1(f)
    def innerLoop2():
        while(True):
            for i in range(len(ofiles)):
                if(mask[i] == False):
                    c = ofiles[i].read(1)
                    if(c == '1' or c == '0'):
                        outputs[i] = c
                        mask[i] = True
                    if all(mask):
                        return
                    if c == '':
                        return
    innerLoop2()                
    eq = eq and (all_equal(outputs))
    while eq and all(map(lambda x : x != '',outputs)):
        for i in range(len(ofiles)):
            l = ofiles[i].read(16*1024*1024)
            outputs[i] = l
        eq = eq and (all_equal(outputs))
    
    for f in ofiles:
        f.close()
        
    return eq
        
        
def time():
    rePatt = re.compile('running time [\d]*\.?[\d]*')
    result = []
    for file in files:
        with open(file) as mN:
            for line in mN:
                if (rePatt.findall(line) != []):
                    result.append(line)
                    break
    return(result)

def time_naive():
    rePatt = re.compile('running time [\d]*\.?[\d]*')
    result = []
    with open("clean_pe_chunk.txt") as mN:
        for line in mN:
            if (rePatt.findall(line) != []):
                result.append(line)
                break
    return(result)


    
# pattern = randomString(19,False)
# print(pattern)
# randomString(1024*1024*100) #if size > 19 PE aborts

# matchPE("sunmnnctxnvkowicoe")
# matchNaive("sunmnnctxnvkowicoe")
# print(test())

rePatt = re.compile('[\d]+\.?[\d]*')
# for line in time():
#     print(rePatt.findall(line))


def checkTimeStaticDisk(size,offset,device,patterns):
    res = []
#     for _ in range(0,10):
#         pattern = randomString(16,False) #if size > 19 PE aborts
#         for _ in range(2):
    for p in patterns:#bytes
        with open('data/pattern.txt', 'wb') as fb:
                 fb.write(p)
        match(p,device,size,offset) #prerun for jit
#         matchNaiveDevice(pattern)
#         if(test()):
        clocks = []
        for line in time():
            clocks.append(rePatt.findall(line))
        entry = [x[0] for x in clocks] #first time is Naive 2nd is PE'ed
        res.append(entry)
#         else:
#             print("Match failed " + pattern)
#             return
    return res

def checkTimeStatic(size,patterns):
    res = []
    randomString(1024*1024*size)
#     for _ in range(0,10):
#         pattern = randomString(16,False) #if size > 19 PE aborts
#         for _ in range(2):
    for p in patterns:
        with open('data/pattern.txt', 'wb') as fb:
                 fb.write(p.encode('ascii'))
        match('data/pattern.txt','data/subject.txt') #prerun for jit
#         matchNaiveDevice(pattern)
#         if(test()):
        clocks = []
        for line in time():
            clocks.append(rePatt.findall(line))
        entry = [x[0] for x in clocks] #first time is Naive 2nd is PE'ed
        res.append(entry)
        with open('check_time_static.txt','w+') as cts:
            cts.write(str(res))
#         else:
#             print("Match failed " + pattern)
#             return
    return res



bad_example = "./fun aaaaabaaaaabaaaa dirty 0"

# test = [[(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '41', '36', '34', '34', '28', '6', '0', '1', '4', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['59', '45', '44', '35', '34', '35', '6', '0', '1', '3', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '35', '42', '32', '30', '6', '0', '1', '5', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['56', '54', '35', '36', '31', '28', '6', '0', '1', '4', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['46', '38', '36', '36', '31', '29', '6', '0', '1', '3', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['42', '35', '35', '35', '31', '28', '6', '0', '1', '3', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['62', '35', '35', '30', '29', '6', '0', '1', '4', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['57', '36', '35', '31', '28', '6', '0', '1', '4', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['55', '38', '36', '36', '30', '28', '6', '0', '1', '3', '0', '0']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['56', '39', '35', '36', '31', '29', '6', '0', '1', '4', '0', '0'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '38', '36', '35', '31', '29', '26', '4', '20', '20', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['48', '37', '36', '36', '31', '29', '26', '4', '18', '17', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['44', '37', '35', '31', '29', '26', '4', '17', '27', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '43', '35', '35', '31', '29', '27', '4', '19', '20', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['46', '37', '36', '36', '31', '29', '26', '4', '18', '16', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['41', '36', '35', '35', '31', '29', '26', '4', '20', '18', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '36', '35', '31', '29', '26', '4', '18', '20', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['58', '36', '36', '31', '28', '26', '4', '18', '24', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['55', '38', '35', '36', '31', '29', '26', '4', '18', '15', '2', '3']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['57', '40', '36', '36', '31', '29', '26', '4', '19', '19', '2', '3'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['47', '39', '36', '35', '31', '28', '51', '8', '39', '41', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['47', '37', '35', '35', '31', '28', '51', '8', '37', '34', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '36', '36', '31', '28', '52', '8', '35', '55', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '38', '36', '35', '30', '28', '51', '8', '38', '41', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['46', '37', '36', '35', '31', '28', '51', '8', '37', '32', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['42', '37', '36', '35', '31', '28', '51', '8', '39', '36', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '46', '35', '32', '29', '52', '8', '37', '40', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['56', '36', '35', '30', '28', '51', '8', '36', '49', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['53', '38', '35', '35', '31', '29', '51', '8', '37', '31', '5', '7']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['56', '39', '36', '35', '31', '28', '51', '8', '38', '39', '5', '7'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '38', '36', '36', '31', '28', '127', '20', '99', '102', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['47', '38', '35', '36', '31', '28', '127', '20', '93', '85', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['44', '35', '35', '30', '28', '128', '20', '90', '138', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['53', '39', '60', '44', '31', '29', '127', '20', '99', '102', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['46', '37', '36', '36', '31', '29', '126', '20', '95', '81', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['43', '36', '35', '36', '31', '28', '128', '20', '102', '89', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '35', '36', '31', '29', '127', '20', '96', '101', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['57', '36', '36', '45', '37', '126', '20', '93', '122', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['53', '38', '38', '37', '31', '28', '126', '20', '96', '78', '14', '17']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['55', '50', '39', '47', '39', '29', '128', '20', '99', '99', '14', '17'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '39', '37', '35', '30', '28', '251', '40', '203', '205', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['50', '39', '37', '36', '31', '29', '253', '40', '191', '171', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '36', '35', '31', '29', '252', '40', '185', '276', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['70', '44', '42', '41', '35', '30', '252', '40', '201', '205', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['50', '37', '36', '37', '34', '29', '253', '40', '190', '163', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['43', '36', '35', '37', '31', '29', '252', '40', '205', '179', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '36', '35', '33', '34', '252', '40', '195', '203', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['57', '38', '37', '33', '29', '248', '40', '191', '245', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['55', '43', '39', '36', '32', '31', '254', '40', '195', '157', '29', '35']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['60', '41', '53', '37', '32', '30', '251', '40', '200', '198', '29', '35'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['46', '38', '37', '37', '31', '29', '377', '61', '304', '308', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '37', '36', '36', '31', '29', '376', '61', '280', '257', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '36', '35', '31', '29', '374', '61', '279', '414', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['51', '39', '36', '36', '31', '29', '378', '61', '306', '308', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['47', '37', '36', '35', '32', '29', '377', '61', '289', '245', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['43', '36', '37', '36', '31', '29', '374', '61', '307', '269', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '36', '36', '31', '29', '382', '61', '297', '305', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['57', '36', '36', '31', '29', '378', '61', '288', '367', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['56', '39', '36', '36', '31', '29', '375', '61', '294', '235', '44', '53']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['56', '40', '36', '36', '32', '30', '384', '61', '300', '297', '44', '53'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['48', '39', '36', '35', '31', '29', '503', '81', '403', '410', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['47', '38', '36', '36', '32', '29', '504', '81', '378', '343', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '35', '35', '38', '29', '509', '81', '370', '552', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['66', '39', '36', '36', '31', '30', '497', '81', '403', '410', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['47', '41', '41', '36', '35', '32', '503', '81', '381', '326', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '36', '37', '38', '32', '30', '507', '81', '408', '360', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['62', '36', '36', '31', '30', '510', '81', '386', '407', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['57', '36', '36', '31', '29', '499', '81', '383', '489', '58', '71']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['57', '38', '36', '35', '33', '29', '510', '81', '392', '313', '58', '70']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['60', '39', '37', '36', '31', '29', '504', '81', '397', '397', '58', '70'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '39', '36', '37', '33', '33', '749', '121', '606', '615', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['50', '41', '37', '37', '31', '30', '754', '121', '571', '515', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['46', '36', '36', '43', '29', '753', '121', '554', '829', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '39', '36', '36', '31', '30', '755', '121', '607', '616', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['47', '37', '37', '36', '31', '30', '765', '122', '574', '490', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['43', '38', '50', '36', '46', '29', '754', '122', '615', '540', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '37', '35', '34', '30', '755', '121', '584', '611', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['58', '37', '41', '34', '29', '753', '122', '577', '734', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['55', '39', '36', '36', '34', '31', '756', '121', '587', '470', '88', '106']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['60', '40', '37', '36', '32', '29', '763', '122', '599', '595', '88', '106'])], [(['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '39', '36', '36', '32', '29', '1003', '162', '803', '821', '117', '142']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['48', '37', '36', '44', '31', '29', '1005', '162', '758', '687', '117', '142']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['45', '36', '36', '31', '29', '998', '162', '738', '1105', '117', '141']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['50', '38', '35', '35', '31', '28', '1004', '162', '805', '822', '117', '142']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['49', '38', '38', '36', '31', '28', '1005', '162', '767', '653', '117', '142']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['44', '36', '35', '35', '32', '29', '1007', '162', '811', '720', '117', '142']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '35', '37', '57', '43', '1004', '162', '777', '814', '117', '142']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['63', '39', '40', '35', '30', '1000', '162', '765', '979', '117', '141']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['61', '59', '37', '62', '53', '53', '999', '163', '772', '627', '117', '141']), (['aaababbbbbbb', 'baaabbababba', 'bbbbbaaabbba', 'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa', 'aabaaabababa', 'bbbbaababbab', 'abbbbbbabbab', 'ababbbbabaab'], ['67', '41', '38', '37', '33', '33', '1010', '162', '796', '794', '117', '142'])]]

# test_data = [[['59', '40', '40', '49', '45', '38', '35', '38', '31', '41', '6', '0', '0', '0'], ['124', '60', '65', '58', '36', '51', '38', '32', '35', '52', '6', '0', '0', '0'], ['49', '45', '67', '67', '50', '45', '46', '65', '47', '75', '6', '0', '0', '0'], ['58', '44', '56', '55', '64', '77', '50', '64', '51', '48', '6', '0', '0', '0'], ['70', '67', '86', '59', '64', '58', '50', '45', '60', '56', '6', '0', '0', '0'], ['98', '66', '92', '99', '67', '71', '63', '66', '46', '59', '6', '0', '0', '0'], ['83', '52', '56', '88', '48', '46', '40', '36', '46', '57', '6', '0', '0', '0']], [['60', '47', '47', '57', '46', '42', '35', '36', '39', '41', '51', '8', '5', '7'], ['59', '48', '46', '62', '48', '49', '38', '35', '39', '47', '51', '8', '5', '7'], ['60', '49', '47', '62', '43', '42', '37', '35', '34', '38', '51', '8', '5', '7'], ['57', '43', '46', '56', '44', '42', '45', '41', '52', '58', '51', '8', '5', '7'], ['63', '56', '55', '70', '61', '59', '48', '51', '48', '52', '51', '8', '5', '7'], ['95', '64', '59', '86', '56', '55', '51', '47', '46', '50', '51', '8', '5', '7'], ['96', '61', '66', '94', '55', '56', '50', '46', '47', '52', '52', '8', '5', '7']], [['86', '64', '62', '70', '56', '54', '56', '54', '79', '51', '128', '20', '14', '17'], ['71', '53', '63', '72', '55', '52', '48', '45', '41', '50', '126', '20', '14', '17'], ['89', '69', '51', '79', '53', '51', '48', '44', '32', '81', '126', '20', '14', '17'], ['45', '36', '37', '45', '35', '35', '30', '28', '29', '32', '128', '20', '14', '17'], ['41', '35', '35', '41', '35', '35', '31', '28', '29', '32', '128', '20', '14', '17'], ['53', '37', '37', '53', '36', '35', '30', '28', '29', '33', '126', '20', '14', '17'], ['55', '38', '38', '55', '35', '35', '33', '28', '28', '32', '127', '20', '14', '17']], [['50', '38', '38', '45', '35', '35', '30', '28', '29', '32', '249', '40', '29', '35'], ['47', '38', '37', '46', '36', '35', '30', '28', '37', '32', '254', '40', '29', '35'], ['49', '38', '38', '49', '35', '35', '30', '28', '29', '32', '253', '40', '29', '35'], ['57', '38', '40', '50', '36', '35', '31', '28', '29', '32', '251', '40', '29', '35'], ['42', '36', '36', '42', '35', '35', '30', '28', '29', '32', '251', '40', '29', '35'], ['54', '38', '38', '54', '35', '35', '30', '28', '29', '33', '254', '40', '29', '35'], ['56', '39', '39', '56', '35', '35', '30', '28', '29', '33', '251', '40', '29', '35']], [['49', '38', '38', '45', '35', '35', '30', '28', '29', '33', '380', '61', '44', '53'], ['47', '37', '37', '47', '35', '35', '31', '28', '30', '32', '378', '61', '44', '53'], ['50', '38', '38', '50', '35', '35', '30', '29', '29', '32', '380', '60', '44', '53'], ['46', '37', '37', '46', '35', '35', '31', '28', '29', '33', '371', '61', '44', '53'], ['42', '35', '36', '42', '37', '36', '30', '28', '29', '32', '376', '61', '44', '53'], ['54', '38', '38', '55', '36', '36', '30', '29', '29', '32', '377', '61', '44', '53'], ['56', '38', '39', '56', '35', '35', '31', '28', '29', '33', '379', '60', '44', '53']], [['45', '38', '39', '45', '36', '35', '31', '28', '29', '34', '500', '81', '58', '70'], ['47', '37', '37', '47', '35', '35', '30', '28', '29', '33', '505', '81', '58', '70'], ['49', '38', '38', '50', '35', '35', '31', '29', '29', '33', '496', '81', '58', '70'], ['46', '37', '37', '60', '35', '35', '31', '29', '29', '32', '502', '81', '58', '70'], ['42', '36', '36', '42', '35', '35', '31', '29', '30', '32', '505', '81', '58', '70'], ['54', '38', '38', '55', '36', '35', '31', '28', '29', '32', '501', '81', '58', '70'], ['56', '39', '40', '56', '36', '36', '31', '29', '29', '33', '507', '81', '58', '71']], [['45', '39', '38', '45', '35', '35', '31', '28', '29', '33', '753', '122', '88', '106'], ['47', '37', '37', '47', '35', '35', '31', '29', '29', '33', '753', '122', '88', '106'], ['49', '38', '38', '49', '35', '36', '31', '29', '30', '33', '751', '122', '88', '106'], ['46', '37', '37', '46', '36', '35', '31', '28', '31', '33', '755', '122', '88', '106'], ['43', '36', '36', '42', '36', '36', '31', '29', '29', '33', '745', '122', '88', '106'], ['54', '38', '38', '54', '36', '36', '31', '28', '29', '33', '757', '122', '87', '106'], ['56', '39', '39', '57', '35', '35', '31', '29', '30', '33', '751', '122', '87', '106']], [['51', '39', '38', '46', '36', '36', '31', '29', '29', '33', '1001', '162', '117', '141'], ['47', '37', '38', '47', '36', '35', '31', '29', '29', '32', '1003', '163', '117', '141'], ['49', '38', '38', '50', '36', '36', '31', '28', '30', '33', '992', '163', '117', '142'], ['46', '37', '38', '47', '36', '36', '31', '29', '31', '33', '1000', '162', '117', '141'], ['42', '36', '36', '42', '35', '36', '30', '29', '29', '33', '1013', '162', '117', '141'], ['55', '38', '38', '54', '35', '37', '31', '29', '29', '33', '1006', '162', '117', '142'], ['56', '39', '39', '56', '35', '35', '31', '29', '30', '33', '1015', '162', '117', '141']], [['50', '38', '39', '45', '36', '36', '31', '29', '29', '33', '2001', '325', '234', '283'], ['47', '37', '37', '47', '36', '35', '31', '29', '29', '33', '2005', '325', '234', '283'], ['49', '39', '38', '49', '35', '36', '31', '29', '29', '33', '2005', '325', '234', '283'], ['46', '37', '37', '47', '35', '36', '31', '29', '30', '33', '2009', '325', '234', '283'], ['43', '37', '36', '42', '35', '36', '31', '29', '29', '32', '1995', '325', '234', '283'], ['55', '38', '39', '55', '36', '36', '31', '29', '29', '33', '2002', '325', '234', '283'], ['56', '39', '39', '57', '36', '36', '31', '29', '30', '33', '2009', '325', '234', '283']], [['48', '38', '38', '45', '35', '35', '31', '28', '29', '32', '3011', '487', '351', '424'], ['47', '37', '37', '47', '35', '35', '31', '28', '29', '33', '3016', '487', '351', '424'], ['49', '38', '38', '49', '36', '35', '31', '29', '29', '32', '3001', '487', '351', '424'], ['46', '45', '37', '46', '35', '35', '31', '28', '29', '32', '3003', '487', '351', '424'], ['42', '36', '36', '42', '35', '35', '31', '29', '29', '32', '3010', '487', '351', '424'], ['54', '38', '38', '54', '35', '36', '31', '29', '30', '33', '3010', '487', '351', '424'], ['56', '39', '39', '56', '35', '35', '31', '28', '29', '33', '3015', '487', '351', '424']], [['47', '40', '38', '46', '37', '35', '30', '29', '30', '32', '3996', '649', '468', '566'], ['66', '37', '39', '49', '37', '35', '30', '29', '29', '33', '4000', '649', '468', '566'], ['49', '40', '38', '49', '36', '36', '31', '29', '29', '33', '3996', '649', '468', '566'], ['46', '38', '39', '46', '35', '36', '30', '28', '29', '32', '3999', '649', '468', '566'], ['42', '36', '36', '42', '36', '35', '31', '29', '29', '34', '4003', '649', '468', '566'], ['54', '38', '39', '54', '36', '36', '31', '29', '29', '32', '4016', '649', '468', '566'], ['56', '39', '39', '56', '37', '35', '31', '28', '30', '36', '4010', '649', '468', '566']], [['48', '39', '39', '45', '35', '35', '31', '29', '29', '33', '5011', '815', '588', '711'], ['47', '37', '38', '47', '35', '36', '31', '28', '29', '33', '5031', '815', '588', '711'], ['49', '38', '38', '50', '35', '35', '32', '29', '29', '32', '5025', '815', '588', '711'], ['46', '37', '37', '46', '35', '35', '31', '28', '30', '33', '4990', '815', '588', '711'], ['42', '36', '36', '42', '35', '35', '31', '29', '29', '32', '5024', '815', '588', '711'], ['55', '38', '38', '55', '35', '35', '31', '28', '29', '33', '5035', '815', '588', '711'], ['56', '39', '39', '56', '35', '35', '31', '29', '29', '33', '5024', '815', '588', '711']], [['48', '39', '38', '46', '35', '35', '31', '28', '30', '33', '5995', '975', '704', '851'], ['47', '37', '37', '47', '35', '36', '31', '29', '30', '34', '6005', '975', '704', '851'], ['49', '38', '38', '51', '35', '35', '31', '29', '30', '33', '6009', '976', '704', '851'], ['46', '37', '37', '46', '36', '35', '31', '29', '29', '33', '6000', '975', '704', '851'], ['42', '36', '36', '43', '35', '35', '31', '29', '29', '33', '5980', '975', '704', '851'], ['54', '38', '39', '54', '35', '35', '31', '28', '29', '33', '6020', '975', '704', '851'], ['100', '39', '39', '57', '54', '37', '31', '29', '44', '33', '6012', '975', '704', '851']], [['48', '39', '39', '48', '37', '35', '31', '80', '38', '34', '7994', '1296', '934', '1129'], ['84', '69', '63', '107', '47', '43', '36', '33', '36', '63', '8019', '1296', '934', '1129'], ['61', '48', '41', '52', '36', '55', '39', '53', '43', '36', '8049', '1296', '934', '1129'], ['69', '58', '55', '78', '86', '36', '31', '46', '48', '33', '8032', '1296', '934', '1129'], ['62', '36', '37', '61', '37', '50', '44', '47', '41', '33', '8018', '1296', '934', '1129'], ['55', '39', '40', '55', '36', '36', '31', '29', '30', '33', '8020', '1296', '934', '1129'], ['57', '41', '39', '57', '54', '36', '32', '29', '46', '45', '8006', '1296', '934', '1129']], [['48', '59', '57', '63', '48', '50', '51', '41', '74', '46', '9271', '1505', '1086', '1312'], ['75', '67', '55', '66', '56', '45', '81', '39', '49', '46', '9289', '1504', '1085', '1312'], ['70', '61', '59', '70', '44', '46', '43', '49', '148', '45', '9269', '1505', '1087', '1313'], ['71', '62', '69', '82', '65', '112', '49', '72', '76', '86', '9268', '1504', '1085', '1311'], ['71', '84', '37', '66', '72', '54', '50', '54', '54', '67', '9266', '1504', '1085', '1311'], ['72', '89', '111', '82', '52', '68', '74', '41', '42', '98', '9246', '1504', '1085', '1311'], ['97', '55', '63', '73', '47', '92', '43', '50', '50', '68', '9239', '1505', '1085', '1311']]]

testData = [[['50', '65', '35', '32', '99', '111', '32', '34', '2', '0', '0', '0'], ['54', '52', '44', '32', '117', '98', '32', '33', '2', '0', '0', '0'], ['75', '54', '35', '36', '100', '101', '31', '33', '2', '0', '0', '0'], ['53', '51', '35', '36', '171', '105', '32', '35', '2', '0', '0', '0'], ['48', '53', '39', '31', '104', '103', '32', '33', '2', '0', '0', '0'], ['63', '54', '34', '37', '112', '113', '32', '45', '2', '0', '0', '0'], ['62', '51', '43', '31', '102', '106', '31', '33', '2', '0', '0', '0']], [['84', '140', '111', '49', '143', '108', '42', '76', '40', '8', '8', '9'], ['85', '152', '87', '41', '116', '120', '60', '78', '40', '8', '8', '9'], ['79', '140', '76', '39', '115', '101', '49', '77', '40', '8', '8', '9'], ['77', '159', '74', '57', '130', '169', '51', '81', '40', '8', '8', '9'], ['78', '205', '137', '74', '227', '189', '45', '83', '40', '8', '8', '9'], ['93', '171', '82', '48', '115', '105', '55', '87', '41', '8', '8', '9'], ['157', '169', '92', '55', '167', '231', '51', '101', '40', '8', '8', '9']], [['92', '278', '155', '113', '241', '128', '70', '160', '104', '20', '20', '22'], ['126', '328', '171', '61', '227', '145', '64', '151', '104', '21', '20', '23'], ['130', '301', '163', '68', '207', '145', '65', '155', '104', '20', '20', '22'], ['131', '351', '155', '67', '217', '206', '105', '149', '104', '21', '20', '22'], ['150', '439', '205', '85', '202', '113', '78', '141', '104', '20', '23', '23'], ['197', '391', '199', '92', '266', '243', '83', '173', '104', '21', '20', '22'], ['182', '303', '169', '62', '216', '109', '53', '138', '104', '20', '20', '22']], [['135', '493', '239', '67', '298', '112', '105', '237', '204', '41', '40', '45'], ['188', '606', '246', '68', '281', '147', '78', '242', '210', '42', '40', '45'], ['135', '530', '259', '70', '284', '152', '71', '238', '211', '41', '40', '45'], ['157', '604', '242', '84', '282', '111', '98', '241', '208', '42', '40', '45'], ['268', '666', '231', '70', '312', '144', '71', '241', '201', '41', '40', '45'], ['162', '640', '249', '81', '274', '125', '75', '264', '207', '42', '40', '45'], ['146', '553', '244', '72', '297', '126', '77', '245', '209', '41', '40', '45']], [['178', '722', '347', '88', '405', '133', '93', '341', '315', '62', '61', '68'], ['195', '878', '341', '88', '380', '133', '93', '349', '303', '63', '61', '68'], ['179', '758', '346', '88', '378', '133', '92', '332', '310', '62', '61', '68'], ['197', '888', '346', '88', '386', '133', '93', '346', '307', '63', '61', '68'], ['185', '981', '337', '88', '381', '134', '92', '344', '306', '62', '61', '68'], ['208', '918', '341', '88', '372', '134', '94', '337', '310', '63', '61', '68'], ['189', '696', '344', '87', '388', '134', '92', '345', '317', '62', '61', '68']], [['216', '950', '423', '107', '457', '162', '113', '435', '402', '83', '81', '90'], ['245', '1128', '432', '107', '487', '158', '113', '428', '407', '84', '81', '91'], ['220', '997', '452', '106', '481', '186', '113', '441', '408', '83', '81', '91'], ['247', '1168', '449', '107', '488', '167', '114', '452', '420', '84', '81', '91'], ['232', '1297', '445', '107', '491', '154', '113', '445', '399', '83', '81', '91'], ['258', '1211', '434', '107', '461', '154', '113', '427', '385', '84', '81', '91'], ['233', '914', '436', '107', '482', '155', '113', '436', '413', '83', '81', '91']], [['301', '1404', '646', '146', '663', '199', '155', '647', '599', '125', '121', '136'], ['341', '1667', '621', '146', '699', '209', '156', '632', '575', '126', '122', '137'], ['305', '1477', '645', '146', '700', '198', '155', '639', '631', '125', '121', '136'], ['346', '1734', '645', '146', '706', '198', '156', '649', '634', '126', '122', '137'], ['326', '1925', '613', '146', '693', '198', '155', '632', '580', '125', '122', '136'], ['359', '1795', '650', '147', '692', '197', '156', '652', '621', '126', '123', '137'], ['321', '1351', '661', '147', '701', '197', '155', '646', '625', '125', '122', '136']], [['385', '1859', '828', '185', '865', '241', '197', '833', '772', '167', '162', '182'], ['439', '2211', '859', '185', '888', '241', '198', '832', '841', '168', '163', '183'], ['388', '1956', '839', '185', '883', '239', '197', '859', '790', '167', '162', '182'], ['445', '2299', '830', '186', '879', '242', '198', '814', '824', '168', '163', '183'], ['419', '2555', '857', '185', '876', '239', '198', '857', '824', '167', '162', '182'], ['460', '2382', '828', '185', '907', '240', '198', '848', '809', '168', '163', '183'], ['405', '1788', '848', '185', '885', '240', '197', '850', '798', '167', '163', '182']], [['716', '3678', '1634', '340', '1662', '412', '363', '1576', '1570', '334', '324', '363'], ['831', '4384', '1673', '341', '1722', '410', '365', '1692', '1584', '336', '326', '366'], ['726', '3871', '1667', '339', '1666', '408', '363', '1697', '1577', '334', '325', '364'], ['845', '4558', '1599', '342', '1746', '410', '366', '1621', '1506', '336', '327', '366'], ['795', '5070', '1617', '339', '1650', '406', '365', '1621', '1633', '334', '325', '364'], ['864', '4726', '1610', '341', '1691', '407', '366', '1682', '1632', '336', '327', '366'], ['752', '3535', '1543', '340', '1695', '406', '364', '1630', '1578', '335', '325', '365']], [['1050', '5498', '2440', '494', '2372', '575', '530', '2337', '2339', '500', '486', '545'], ['1222', '6558', '2503', '498', '2550', '575', '533', '2514', '2447', '504', '489', '549'], ['1062', '5788', '2309', '494', '2443', '573', '531', '2387', '2308', '501', '487', '546'], ['1241', '6820', '2482', '498', '2541', '575', '533', '2518', '2478', '504', '490', '549'], ['1171', '7586', '2386', '495', '2387', '574', '531', '2441', '2383', '502', '487', '546'], ['1265', '7066', '2216', '498', '2433', '576', '534', '2247', '2362', '504', '490', '550'], ['1098', '5282', '2455', '495', '2472', '574', '531', '2466', '2362', '502', '488', '547']], [['1383', '7320', '3234', '648', '3317', '740', '697', '3240', '3208', '667', '649', '727'], ['1612', '8731', '3063', '653', '3228', '743', '702', '3182', '3189', '672', '653', '732'], ['1397', '7704', '3263', '649', '3187', '740', '697', '3167', '3190', '668', '649', '728'], ['1638', '9082', '3177', '655', '3243', '744', '702', '3215', '3263', '673', '654', '733'], ['1546', '10105', '3077', '652', '3298', '742', '698', '3240', '3195', '669', '650', '729'], ['1670', '9408', '3193', '654', '3330', '745', '702', '3243', '3158', '673', '654', '733'], ['1442', '7030', '3237', '651', '3320', '743', '699', '3231', '3155', '670', '651', '730']], [['1712', '9134', '3873', '804', '3969', '911', '867', '3855', '3786', '837', '814', '911'], ['2000', '10902', '3504', '810', '3990', '915', '872', '3893', '3925', '842', '819', '917'], ['1734', '9618', '3664', '805', '3990', '910', '868', '3982', '4088', '838', '814', '912'], ['2037', '11342', '3737', '810', '4026', '918', '874', '3919', '3751', '843', '820', '918'], ['1922', '12620', '3709', '807', '3924', '912', '869', '3944', '3743', '839', '815', '913'], ['2071', '11752', '3981', '811', '4007', '917', '874', '3810', '3745', '844', '820', '919'], ['1791', '8804', '3903', '807', '4129', '915', '870', '3881', '3801', '840', '816', '915']], [['2044', '10962', '4538', '961', '4891', '1105', '1033', '4677', '4743', '1003', '975', '1092'], ['2393', '13078', '4762', '967', '4833', '1085', '1039', '4856', '4790', '1009', '981', '1099'], ['2072', '11554', '4673', '961', '4811', '1081', '1054', '4823', '4753', '1004', '975', '1093'], ['2443', '13602', '4524', '968', '4750', '1087', '1041', '4767', '4518', '1010', '982', '1100'], ['2320', '15164', '4671', '963', '4888', '1083', '1035', '4811', '4600', '1005', '977', '1094'], ['2497', '14097', '4632', '968', '4726', '1085', '1041', '4761', '4648', '1011', '982', '1101'], ['2157', '10525', '4666', '975', '4782', '1094', '1037', '4810', '4684', '1006', '978', '1096']], [['2732', '14600', '5632', '1277', '6366', '1408', '1374', '6320', '6134', '1331', '1293', '1450'], ['3198', '17445', '5920', '1293', '6446', '1416', '1382', '6456', '6108', '1340', '1302', '1459'], ['2755', '15409', '6014', '1288', '6404', '1411', '1376', '6324', '6192', '1332', '1295', '1451'], ['3248', '18131', '6396', '1303', '6361', '1418', '1384', '6451', '6270', '1341', '1303', '1461'], ['3063', '20181', '6125', '1276', '6423', '1417', '1379', '6262', '6318', '1334', '1297', '1453'], ['3302', '18786', '6069', '1303', '6298', '1418', '1385', '6380', '5994', '1342', '1304', '1462'], ['2844', '14023', '6261', '1291', '6305', '1411', '1379', '6349', '6263', '1336', '1298', '1455']]]

testDataKmp = [[['51', '90', '141', '2', '1'], ['55', '98', '142', '2', '1'], ['55', '89', '213', '2', '1'], ['54', '85', '140', '2', '1'], ['50', '85', '139', '2', '1'], ['59', '83', '143', '2', '1'], ['62', '95', '152', '2', '1']], [['72', '295', '141', '40', '38'], ['71', '316', '138', '40', '36'], ['68', '305', '203', '40', '39'], ['141', '324', '224', '40', '36'], ['84', '331', '198', '40', '39'], ['109', '304', '197', '40', '38'], ['109', '324', '198', '40', '38']], [['113', '665', '229', '103', '100'], ['116', '641', '230', '104', '95'], ['109', '662', '227', '103', '99'], ['117', '638', '227', '104', '95'], ['113', '675', '218', '104', '102'], ['127', '624', '219', '104', '97'], ['117', '657', '232', '103', '99']], [['160', '1237', '298', '208', '202'], ['161', '1194', '293', '211', '191'], ['150', '1249', '306', '203', '199'], ['147', '1170', '265', '205', '194'], ['148', '1278', '394', '209', '205'], ['190', '1206', '303', '206', '196'], ['171', '1244', '324', '209', '202']], [['208', '1837', '429', '305', '305'], ['234', '1804', '425', '311', '281'], ['203', '1807', '369', '315', '302'], ['210', '1729', '333', '319', '288'], ['246', '1938', '508', '313', '309'], ['353', '1774', '550', '317', '295'], ['190', '1796', '340', '316', '301']], [['316', '2457', '1035', '405', '402'], ['322', '2456', '531', '411', '385'], ['369', '2427', '523', '420', '397'], ['360', '2328', '495', '418', '387'], ['273', '2501', '583', '405', '410'], ['282', '2363', '431', '415', '391'], ['355', '2456', '655', '413', '396']], [['303', '3580', '630', '604', '609'], ['360', '3528', '560', '600', '562'], ['328', '3623', '747', '611', '605'], ['374', '3416', '609', '600', '566'], ['352', '3673', '654', '611', '614'], ['412', '3362', '574', '613', '591'], ['414', '3611', '584', '627', '599']], [['402', '4777', '742', '811', '797'], ['452', '4628', '704', '819', '753'], ['400', '4801', '765', '839', '812'], ['474', '4567', '732', '784', '763'], ['452', '4818', '719', '779', '818'], ['487', '4455', '713', '803', '772'], ['408', '4704', '723', '845', '791']], [['713', '9460', '1351', '1546', '1608'], ['852', '9093', '1280', '1621', '1525'], ['744', '9413', '1513', '1611', '1597'], ['911', '8961', '1288', '1619', '1526'], ['810', '9555', '1394', '1617', '1621'], ['888', '8895', '1346', '1570', '1559'], ['764', '9352', '1329', '1566', '1598']], [['1207', '14113', '2133', '2317', '2407'], ['1232', '13650', '1912', '2390', '2259'], ['1079', '14118', '2014', '2315', '2403'], ['1257', '13420', '1999', '2315', '2116'], ['1207', '14314', '2000', '2425', '2428'], ['1282', '13260', '1889', '2375', '2326'], ['1098', '14002', '2014', '2349', '2386']], [['1429', '18824', '2676', '3093', '3199'], ['1633', '18113', '2523', '3057', '3046'], ['1406', '18764', '2618', '3073', '3190'], ['1663', '17871', '2590', '3181', '3053'], ['1608', '19080', '2589', '3133', '3247'], ['1695', '17651', '2494', '3298', '3083'], ['1448', '18655', '2593', '3313', '3175']], [['1848', '23516', '3305', '3770', '3993'], ['2032', '22607', '3160', '4018', '3803'], ['1756', '23496', '3338', '3663', '4009'], ['2122', '22333', '3076', '4033', '3796'], ['1962', '23841', '3215', '3606', '4046'], ['2095', '22068', '3080', '3956', '3895'], ['1861', '23332', '3267', '4056', '4007']], [['2114', '28195', '4065', '4714', '4794'], ['2522', '27130', '3698', '4471', '4414'], ['2184', '28163', '3976', '4474', '4796'], ['2485', '26868', '3830', '4533', '4475'], ['2312', '28599', '3994', '4540', '4850'], ['2579', '26572', '3751', '4615', '4625'], ['2231', '28065', '3875', '4650', '4774']], [['2848', '37610', '5145', '5944', '6405'], ['3237', '36236', '5531', '6352', '6054'], ['2797', '37458', '5001', '6338', '6404'], ['3276', '35756', '4780', '6320', '5923'], ['3084', '38034', '5099', '6171', '6481'], ['3342', '35352', '4923', '5967', '6209'], ['2856', '37259', '5029', '5973', '6363']]]

testDataNaivePe = [[['507', '41', '40'], ['113', '42', '40'], ['106', '41', '40'], ['68', '42', '40'], ['70', '41', '40'], ['72', '42', '40'], ['76', '42', '41']], [['120', '62', '61'], ['113', '63', '61'], ['93', '62', '61'], ['95', '63', '61'], ['94', '63', '61'], ['95', '63', '61'], ['92', '63', '61']], [['115', '83', '81'], ['111', '84', '81'], ['111', '83', '81'], ['107', '84', '81'], ['106', '83', '81'], ['109', '84', '81'], ['118', '83', '81']], [['146', '125', '121'], ['146', '126', '122'], ['146', '125', '121'], ['147', '126', '122'], ['161', '125', '122'], ['171', '126', '122'], ['151', '125', '122']], [['212', '167', '162'], ['204', '168', '163'], ['246', '167', '162'], ['211', '168', '163'], ['203', '168', '163'], ['261', '168', '163'], ['199', '167', '163']], [['371', '334', '325'], ['427', '336', '326'], ['354', '334', '325'], ['409', '336', '327'], ['366', '335', '325'], ['440', '337', '327'], ['380', '335', '326']], [['532', '501', '487'], ['516', '504', '489'], ['682', '501', '487'], ['522', '504', '490'], ['507', '502', '488'], ['513', '505', '490'], ['577', '502', '488']], [['847', '668', '649'], ['666', '672', '653'], ['663', '668', '649'], ['669', '673', '654'], ['663', '669', '650'], ['669', '673', '654'], ['667', '670', '651']], [['820', '837', '814'], ['886', '843', '819'], ['834', '838', '815'], ['905', '844', '820'], ['894', '839', '816'], ['886', '844', '820'], ['826', '840', '816']], [['997', '1003', '975'], ['1199', '1009', '981'], ['1020', '1004', '976'], ['1108', '1011', '982'], ['982', '1005', '977'], ['999', '1011', '983'], ['1012', '1007', '978']], [['1284', '1331', '1294'], ['1315', '1339', '1301'], ['1296', '1332', '1295'], ['1349', '1341', '1304'], ['1321', '1334', '1296'], ['1310', '1342', '1304'], ['1277', '1336', '1298']], [['1445', '1501', '1459'], ['1452', '1510', '1468'], ['1449', '1503', '1460'], ['1508', '1513', '1470'], ['1454', '1505', '1463'], ['1449', '1513', '1471'], ['1447', '1506', '1464']]]
# patterns = [randomString(12,False) for i in range(0,10)]
#precompiled set of patterns
patterns = ['aaababbbbbbb', 'baaabbababba',
#             'bbbbbaaabbba',
            'bbbaaabbabbb', 'baaaabbabbab', 'aababababbaa',
#             'aabaaabababa',
#             'bbbbaababbab',
            'abbbbbbabbab', 'ababbbbabaab']
sizes = [50,75,100,150,200,400,600,800,1000,1200,1600,1800]
# ,75,100,150,200]

patterns_pe = ['bbbbabaaaaaabaababbabbbaaaabbaaabaabbabbbbbbbababbaaababbbaabbba', 'abaaabbbbaaaaaababbaabbabbaabbbbbaababbaaaabaabaabbabbaaabbbbbaa',
               'abaabbbbbaabbbbbbabbbbbbaababbaabbbbaaaaaabbbaaaaaababbbababaabb', 'bbbbbbbbbbbabbababaabaaabbabbbaaabbaaababbbbaabbbbbabbababbbabba',
               'bbbbbbaaabaaabaaaabaabbbaaababbbabaaabbbbaababaaabababaabaaabbab', 'aaabbbbababbbabaaabaabaaabaaababbabbaabbbbabbbaaaaabbaaaabbbabba',
               'aaabaaaaabbbaabbabbbaaaaaabbabbaabbbababbbbabbababbabaababaabbba', 'babbbbbabbaaabbaababaabaabbaabbababaaabaaaabaabaaababbaaabbaaabb',
               'aababaabbabbbabaabbaabbbababbbababaaabaaababbabababbaaaaaaababba', 'bbbaaababbbaabaabbbabaabbbabbbbaaaabaaaaabbbaabaabaaababbbbababb']

# for s in sizes:
#     testDataNaivePe.append(checkTimeStatic(s,patterns))

realWorldPatterns = [b'\x1A\x45\xDF\xA3\x93\x42\82\x88\x6D\x61\x74\x72\x6F\x73\x6B\x61', #MKV
                     b'\x25\x50\x44\x46',#PDF
                     b'\x37\x7A\xBC\xAF\x27\x1C', #7-zip
                     b'\x38\x42\x50\x53', #PSD
                     b'\x50\x4B\x03\x04uy',#ZIP
                     b'\x89\x50\x4E\x47\x0D\x0A\x1A\x0A', #PNG
                     b'\xD0\xCF\x11\xE0\xA1\xB1\x1A\xE1', #DOC
                     b'\x4D\x5A', #DLL
                     b'\x3C\x21\x64\x6F\x63\x74\x79\x70', #HTML
                     b'\x1F\x8B\x08', #TAR?
                     b'\x66\x74\x79\x70\x4D\x53\x4E\x56',
                    ]

p = [x[:4] for x in patterns]
# print(testDataNaivePe)
# for p_ in p:
# match('aaa','data/subject.txt')
# subprocess.call(["./clean.out > cuda_naive_chunk.txt],[--pattern=$'\x97\x97'], [--filename=data/subject.txt], [--algorithm=naive], [--type=0], [--size=0], [--offset=0]",shell=True)
# def diskSize():
#     with open('/dev/sda','r') as d:
#         return d.read(124)
# match(patterns[0])
# print(testDataKmp)
# print(patterns_pe)
# print(diskSize())
# TODO TEST AND CHECK ALL ALGS + RUN BENCHMARKS + KMP
# randomString(1*1024*1024)
print(checkTimeStatic(1,patterns))
# with open('data/pattern.txt', 'wb') as fb:
#                  fb.write(patterns[0][:4].encode('ascii'))
# /match('data/pattern.txt', 'data/subject.txt')
# print(test())
# match('\x89\x50\x4E\x47\x0D\x0A\x1A\x0A','data/images.png')
                 

[['1136', '193', '2', '0', '0', '2', '2', '1', '1'], ['296', '169', '2', '0', '0', '2', '2', '1', '1'], ['294', '176', '2', '0', '0', '2', '2', '1', '1'], ['296', '168', '2', '0', '0', '2', '2', '1', '1'], ['289', '184', '2', '0', '0', '2', '2', '1', '1'], ['292', '171', '2', '0', '0', '2', '2', '1', '1'], ['290', '166', '2', '0', '0', '2', '2', '1', '1']]
